In [1]:
from __future__ import absolute_import, division, print_function
import pandas as pd
# load the data into panda dataframe
data_file_name = "Health_and_Personal_Care_5.json"
raw_df = pd.read_json(data_file_name, lines=True)
print("Data loaded")

Data loaded


In [2]:
# View information about the data
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346355 entries, 0 to 346354
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   reviewerID      346355 non-null  object
 1   asin            346355 non-null  object
 2   reviewerName    343304 non-null  object
 3   helpful         346355 non-null  object
 4   reviewText      346355 non-null  object
 5   overall         346355 non-null  int64 
 6   summary         346355 non-null  object
 7   unixReviewTime  346355 non-null  int64 
 8   reviewTime      346355 non-null  object
dtypes: int64(2), object(7)
memory usage: 23.8+ MB


In [3]:
# Convert all the review text into a long string and print its length
raw_corpus = u''.join(raw_df['reviewText']+" ")
print("Raw Corpus contains {0:,} characters".format(len(raw_corpus)))

Raw Corpus contains 178,581,273 characters


In [4]:
# import natural language toolkit
import nltk
# download the punkt tokenizer
nltk.download('punkt')
print("The punkt tokenizer is downloaded")

The punkt tokenizer is downloaded


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\i5\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Load the punkt tokenizer
tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")
print("The punkt tokenizer is loaded")
# we tokenize the raw string into raw sentences
raw_sentences = tokenizer.tokenize(raw_corpus)
print("We have {0:,} raw sentences".format(len(raw_sentences)))

The punkt tokenizer is loaded
We have 1,824,643 raw sentences


In [6]:
import re
# Clean and split sentence into words
def clean_and_split_str(string):
    strip_special_chars = re.compile("[^A-Za-z]+")
    string = re.sub(strip_special_chars, " ", string)
    return string.strip().split()

In [7]:
# clean each raw sentences and build the list of sentences
sentences = []
for raw_sent in raw_sentences:
    if len(raw_sent) > 0:
        sentences.append(clean_and_split_str(raw_sent))
print("We have {0:,} clean sentences".format(len(sentences)))

We have 1,824,643 clean sentences


In [8]:
print(raw_sentences[30])
print()
print(sentences[30])

I use this magnifier to inspect seeds and leaves.

['I', 'use', 'this', 'magnifier', 'to', 'inspect', 'seeds', 'and', 'leaves']


In [9]:
token_count = sum([len(sentence) for sentence in sentences])
print("The dataset corpus contains {0:,} tokens".format(token_count))

The dataset corpus contains 33,476,197 tokens


In [10]:
import multiprocessing

#Dimensionality of the resulting word vectors
num_features = 300

#Minimum word count threshold
min_word_count = 3

#Number of threads to run in parallel
num_workers = multiprocessing.cpu_count()

#Context window length
context_size = 7

#Seed for the RNG, to make the result reproducible
seed = 1

In [11]:
pip install gensim --upgrade

Requirement already up-to-date: gensim in c:\users\i5\anaconda3\lib\site-packages (4.0.1)
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied, skipping upgrade: smart-open>=1.8.1 in c:\users\i5\anaconda3\lib\site-packages (from gensim) (5.1.0)



In [17]:
import multiprocessing

#Dimensionality of the resulting word vectors
num_features = 300

#Minimum word count threshold
min_word_count = 3

#Number of threads to run in parallel
num_workers = multiprocessing.cpu_count()

#Context window length
context_size = 7

#Seed for the RNG, to make the result reproducible
seed = 1

import gensim
from gensim.models import Word2Vec

word2vec_model = gensim.models.word2vec.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers, 
    size=num_features, 
    min_count=min_word_count, 
    window=context_size)
word2vec_model.build_vocab(sentences=sentences)
print("The vocabulary is built")
print("Word2Vec vocabulary length: ", len(word2vec_model.wv.vocab))

#Start training the model
word2vec_model.train(sentences, total_examples=3, epochs = 5)
print("Training finished")

#Save the model
word2vec_model.save("word2vec_model_trained_on_Health_and_Personal_Care_5.w2v")
print("Model saved")

The vocabulary is built
Word2Vec vocabulary length:  62972
Training finished
Model saved


# https://phdstatsphys.wordpress.com/2018/12/27/word2vec-how-to-train-and-update-it/ - article, helped to solve issues with training